In [1]:
from datetime import date
import ast
import pandas as pd

pd.set_option('display.max_columns', None)
df = catalog.load('raw/openaire/researchproduct')

                    INFO     Loading data from raw/openaire/researchproduct (CSVDataset)...     ]8;id=336578;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=315197;file:///root/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

## Paso 0: Seleccionar columnas con identificador y 'measure'

In [2]:
df_researchproduct = df.loc[:, ['dri:objIdentifier', 'journal']]
df_researchproduct = df_researchproduct.convert_dtypes()

In [3]:
df_researchproduct.head(5)

,dri:objIdentifier,journal
0,doi_dedup___::2577d684a839dc78be3e1307914cdfd5,"{'@eissn': '2624-960X', '@ep': '134', '@sp': '..."
1,doi_________::01bdfeebe25730711be433245a4426e7,"{'@issn': '1867-3880', '@eissn': '1867-3899', ..."
2,doi_dedup___::fccba469c4dbc14c5cec34e69ab21625,"{'@issn': '0273-1177', '@ep': '2000', '@sp': '..."
3,doi_dedup___::000d1effb7dd214508cfa2db791e3814,"{'@issn': '2256-4004', '@eissn': '2539-147X', ..."
4,doi_dedup___::387324dfb1abac93013eccfd545780c1,Blucher Design Proceedings


## Paso 1: Asegurarse de que 'journal' sea un diccionario o lista

In [4]:
df_researchproduct['journal'] = df_researchproduct['journal'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_researchproduct['journal'] = df_researchproduct['journal'].apply(lambda x: [x] if not isinstance(x, list) else x)

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Blucher Design Proceedings                                                                       │
│         ▲                                                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

In [ ]:
df_researchproduct

## Paso 2: Explode la columna 'journal' y reinicia el índice

In [ ]:
df_researchproduct = df_researchproduct.explode('journal').reset_index(drop=True)

In [ ]:
df_researchproduct

## Paso 3: Normalizar la columna 'measure' en nuevas columnas

In [ ]:
journal_expanded = pd.json_normalize(df_researchproduct["journal"])

In [ ]:
journal_expanded

## Paso 4: Concatenar df_researchproduct con df_journal asegurando que los índices están alineados

In [ ]:
df_researchproduct2journal = pd.concat([df_researchproduct, journal_expanded], axis=1)
df_researchproduct2journal.drop(columns='journal', inplace=True)

In [ ]:
df_researchproduct2journal

## Paso 5: Agrego load_datetime

In [ ]:
df_researchproduct2journal['load_datetime'] = date.today()

In [ ]:
df_researchproduct2journal